In [217]:
import os
# CUR_PATH = os.path.dirname(os.path.realpath(__file__))

from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE

PPT_PATH = "sample.pptx"
prs = Presentation(PPT_PATH)

In [218]:

def apply_replacements(shape, replace_dict, letter_spacing=200):
    if not shape.has_text_frame:
        return
    tf = shape.text_frame
    text = tf.text

    for key, val in replace_dict.items():
        if not val:
            continue
        if isinstance(val, dict) and key in text:
            tf.clear()
            for domain, items in val.items():
                # 도메인 헤딩
                p_dom = tf.add_paragraph()
                p_dom.text = domain
                for run in p_dom.runs:
                    run.font.bold = True
                    run.font.spacing = letter_spacing

                # 아이템 불릿
                for item in items:
                    p_item = tf.add_paragraph()
                    p_item.text = item
                    p_item.level = 1
                    p_item.bullet = True
                    for run in p_item.runs:
                        run.font.spacing = letter_spacing

                # 빈 줄
                p_blank = tf.add_paragraph()
                p_blank.text = " "
                p_blank.level = 0
                p_blank.bullet = False

            return

    # --- 2) list 플레이스홀더 처리 ---
    for key, val in replace_dict.items():
        if not val:
            continue
        if isinstance(val, (list, tuple)) and key in text:
            tf.clear()
            # 첫 아이템
            p0 = tf.add_paragraph()
            p0.text = val[0]
            p0.bullet = True
            # 나머지
            for item in val[1:]:
                p = tf.add_paragraph()
                p.text = item
                p.level = p0.level
                p.bullet = True
            return

    # --- 3) 나머지 문자열 플레이스홀더는 run 단위로 replace ---
    for paragraph in tf.paragraphs:
        for run in paragraph.runs:
            for key, val in replace_dict.items():
                if isinstance(val, str) and key in run.text:
                    run.text = run.text.replace(key, val)


def replace_in_presentation(prs, replace_dict):
    for slide in prs.slides:
        for shape in slide.shapes:
            if shape.shape_type == MSO_SHAPE_TYPE.GROUP:
                for sub in shape.shapes:
                    apply_replacements(sub, replace_dict)
            else:
                apply_replacements(shape, replace_dict)


## 들어가는 내용
### 한 번만 들어가는 내용
applicant_name,
job,
phone_number,
slogan,
experience,
award,
vision_slogan,
vision_description,
### 여러 번 들어가는 내용
project_name,
outline,
detail_content_title,
detail_content,
tech_domain,
tech,

In [ ]:
# data_dict = {
#     "applicant_name":"권기범",
#     "job":"AI 개발자",
#     "phone_number":"010-2464-5954",
#     "slogan_main":"AI 개발자 시켜주세요",
#     "experience_list":["이거 해봄", "이거 해봄2", "이거 해봄3"],
#     "award":["상 탔음", "상 탔음2"],
#     "vision_slogan":"돈 많이 벌거임",
#     "vision_description":"돈 많이 주세요",
#     "project_name":"탄소 배출량 관리 시스템",
#     "outline":"개요입니다.",
#     "detail_content_title":"구현한 내용",
#     "detail_content_list":["탄소 배출량 예측 모델", "Django  백엔드"],
#     "tech_section":{
#         "AI": ["pytorch", "huggingface"],
#         "Backend": ["Django"]
#     },
#     "tech":"pytorch"
# }

data_dict = {
    "applicant_name":"권기범",
    "job":"AI 개발자",
    "phone_number":"010-2464-5954",
    "slogan_main": "환경을 생각하는 데이터 기반 개발자",
    "experience_list": [
        "탄소 배출 관리 시스템 개발 경험",
        "Django 백엔드 구축 및 운영",
        "도커 기반 배포 자동화 경험",
        "API 설계 및 문서화"
    ],
    "award_list": [],
    "vision_slogan": "지속가능한 IT 개발로 친환경 세상을 만들다",
    "vision_description": "탄소 배출 관리 프로젝트를 통해 얻은 데이터 처리 및 자동화 역량을 바탕으로, 환경과 사회에 긍정적인 영향을 미치는 지속가능한 소프트웨어 개발에 기여하고자 합니다.",
    "project_name": "co2-emission-management",
    "outline": "탄소 배출량 관리 및 모니터링 시스템",
    "detail_content_list": [
        "Django 서버 개발 및 유지보수",
        "Docker 기반 배포 자동화 구성",
        "MySQL 데이터베이스 설계 및 관리",
        "API 서버 및 Swagger 문서화 작업"
    ],
    "tech_section": {
        "Backend": ["Django", "MySQL"],
        "DevOps": ["Docker", "Swagger"]
    }
}

In [220]:
replace_in_presentation(prs, data_dict)

In [221]:
prs.save('./test.pptx')